In [ ]:
# package install
import argparse
import random

import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
# 학습환경 설정
args = {
    # random seed value
    "seed": 1234,
    # number of epoch
    "n_epoch": 200,
    # number of batch
    "n_batch": 2,
    # learning rate
    "lr": 0.001,
    # weights 저장 위치
    "save_path": "01-02-sentence-prediction.pth",
    # CPU 또는 GPU 사용여부 결정
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}
args = argparse.Namespace(**args)

print(args)

In [ ]:
# random seed 설정
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

In [ ]:
# 입력 문장
raw_inputs = [
    "나는 학생 입니다",
    "나는 좋은 선생님 입니다",
    "당신은 매우 좋은 선생님 입니다"
]

In [ ]:
# 정답: 학생(1), 기타(0)
raw_labels = [1, 0, 0]

## Vocab 생성예제

In [ ]:
words = []
for s in raw_inputs:
    words.extend(s.split())

print(words)

In [ ]:
# 중복단어 제거
words = list(dict.fromkeys(words))

print(words)

In [ ]:
# 각 단어별 인덱스 추가
word_to_id = {"[PAD]": 0, "[UNK]": 1}
for w in words:
    word_to_id[w] = len(word_to_id)

print(word_to_id)

In [ ]:
# 각 인덱스 별 단어
id_to_word = {i: w for w, i in word_to_id.items()}

print(id_to_word)

In [ ]:
# 입력 데이터 index로 변경

inputs = []
for s in raw_inputs:
    inputs.append([word_to_id[w] for w in s.split()])
    
print(inputs)

In [ ]:
# 정답 데이터
labels = raw_labels
print(labels)

In [ ]:
class SimpleDataSet(torch.utils.data.Dataset):
    """ 데이터셋 클래스 """

    def __init__(self, inputs, labels):
        """
        생성자
        :param inputs: 입력
        :param labels: 정답
        """
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        """ 데이터셋 길이 """
        assert len(self.inputs) == len(self.labels)
        return len(self.labels)

    def __getitem__(self, index):
        """
        데이터 한 개 조회
        :param index: 데이터 위치
        """
        return (
            torch.tensor(self.inputs[index]),
            torch.tensor(self.labels[index]),
        )

    def collate_fn(self, batch):
        """
        batch단위로 데이터 처리
        :param batch: batch 단위 데이터
        """
        inputs, labels = list(zip(*batch))

        inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
        labels = torch.stack(labels)

        return [
            inputs,
            labels,
        ]


In [ ]:
# dataset
dataset = SimpleDataSet(inputs, labels)

print(len(dataset))
print(dataset[2])

In [ ]:
# random sample data
sampler = torch.utils.data.RandomSampler(dataset)

train_loader = torch.utils.data.DataLoader(dataset, batch_size = args.n_batch, sampler = sampler, collate_fn = dataset.collate_fn)

In [ ]:
# dataset
dataset = SimpleDataSet(inputs, labels)
# valid loader
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=args.n_batch, sampler=None, collate_fn=dataset.collate_fn)

In [ ]:
t_inputs = torch.tensor(inputs[:1])
t_labels = torch.tensor(labels[:1])

print(t_inputs, t_labels)

In [ ]:
# 단어 일련번호를 단어 벡터로 변환
embed = torch.nn.Embedding(len(word_to_id), 4)
hidden = embed(t_inputs)

print(hidden.shape, hidden)

In [ ]:
# 단어를 특징 중 최대값을 문장의 특징으로 사용
hidden, _ = torch.max(hidden, dim=1)

print(hidden.shape, hidden)

In [ ]:
# 문장 학생(1) 또는 기타(0) 예측
linear = torch.nn.Linear(4, 2)
logits = linear(hidden)

print(logits.shape, logits)

## 모델링

In [ ]:
class SentencePrediction(torch.nn.Module):
    def __init__(self, n_vocab):
        super().__init__()
        self.embed = torch.nn.Embedding(n_vocab, 4)
        self.linear = torch.nn.Linear(4, 2)

    def forward(self, inputs):
        hid = self.embed(inputs)
        hid, _ = torch.max(hid, dim = 1)
        return self.linear(hid)